## Analyze data preprocessed using Short-time Fourier transform (STFT) and create several preprocessed datasets with different hyperparameters (frame size)

In [1]:
from src.preprocessing.data_preprocessing import generate_note_labels
from src.preprocessing.data_preprocessing import calculate_stft_stats
from src.preprocessing.data_preprocessing import process_files_using_stft

In [2]:
from src.preprocessing.data_utils import find_note_range
from src.preprocessing.data_utils import find_max_length
from src.preprocessing.data_utils import pad_data
from src.preprocessing.data_utils import save_preprocessed_data

In [3]:
# Paths to the directories
train_data_dir = '../Data/raw/musicnet/train_data'
train_labels_dir = '../Data/raw/musicnet/train_labels'
test_data_dir = '../Data/raw/musicnet/test_data'
test_labels_dir = '../Data/raw/musicnet/test_labels'

### Explore notes range in dataset

In [4]:
all_label_paths = [train_labels_dir, test_labels_dir]
min_note_value, max_note_value = find_note_range(all_label_paths)
note_range = max_note_value - min_note_value

In [5]:
print('Min note value:', min_note_value)
print('Max note value:', max_note_value)
print('Note range:', note_range)

Min note value: 21
Max note value: 105
Note range: 84


### Create preprocessed dataset with frame size = 1024

#### Initially process audio files to spectrograms and generate labels

In [6]:
train_mean, train_std_dev = calculate_stft_stats(train_data_dir, frame_size=1024, data_size=320)
X_train, y_train = process_files_using_stft(train_data_dir, train_labels_dir, generate_note_labels, mean=train_mean, std=train_std_dev, frame_size=1024, data_size=320)

test_mean, test_std_dev = calculate_stft_stats(test_data_dir, frame_size=1024, data_size=10)
X_test, y_test = process_files_using_stft(test_data_dir, test_labels_dir, generate_note_labels, mean=test_mean, std=test_std_dev, frame_size=1024, data_size=10)

Calculating STFT stats: 100%|██████████| 320/320 [02:01<00:00,  2.64it/s]
Processing files in '../Data/raw/musicnet/train_data' and '../Data/raw/musicnet/train_labels': 100%|██████████| 320/320 [03:11<00:00,  1.67it/s]
Calculating STFT stats: 100%|██████████| 10/10 [00:01<00:00,  6.83it/s]
Processing files in '../Data/raw/musicnet/test_data' and '../Data/raw/musicnet/test_labels': 100%|██████████| 10/10 [00:02<00:00,  4.42it/s]


In [7]:
print('Number of training examples:', len(X_train))
print('Number of training labels:', len(y_train))
print(f"X_train[0] shape: {X_train[0].shape}")
print(f"y_train[0] shape: {y_train[0].shape}")


Number of training examples: 320
Number of training labels: 320
X_train[0] shape: (38507, 98)
y_train[0] shape: (38507, 88)


In [8]:
for i, sample in enumerate(X_train):
    print(f"Sample {i + 1} shape: {sample.shape}")

Sample 1 shape: (38507, 98)
Sample 2 shape: (21632, 98)
Sample 3 shape: (38286, 98)
Sample 4 shape: (31743, 98)
Sample 5 shape: (47030, 98)
Sample 6 shape: (28047, 98)
Sample 7 shape: (61578, 98)
Sample 8 shape: (42244, 98)
Sample 9 shape: (79632, 98)
Sample 10 shape: (59981, 98)
Sample 11 shape: (67543, 98)
Sample 12 shape: (40971, 98)
Sample 13 shape: (31242, 98)
Sample 14 shape: (19794, 98)
Sample 15 shape: (31998, 98)
Sample 16 shape: (61207, 98)
Sample 17 shape: (40345, 98)
Sample 18 shape: (56466, 98)
Sample 19 shape: (51708, 98)
Sample 20 shape: (37749, 98)
Sample 21 shape: (58278, 98)
Sample 22 shape: (36810, 98)
Sample 23 shape: (25054, 98)
Sample 24 shape: (55760, 98)
Sample 25 shape: (35958, 98)
Sample 26 shape: (26476, 98)
Sample 27 shape: (49804, 98)
Sample 28 shape: (62541, 98)
Sample 29 shape: (54221, 98)
Sample 30 shape: (44203, 98)
Sample 31 shape: (39736, 98)
Sample 32 shape: (27895, 98)
Sample 33 shape: (30279, 98)
Sample 34 shape: (40723, 98)
Sample 35 shape: (43488

#### Pad data to the same length

In [9]:
max_length = max(find_max_length(X_train), find_max_length(X_test))
print('Max length:', max_length)

Max length: 92079


In [ ]:
X_train_padded = pad_data(X_train, max_length)
y_train_padded = pad_data(y_train, max_length)
X_test_padded = pad_data(X_test, max_length)
y_test_padded = pad_data(y_test, max_length)

In [11]:
print('X_train_padded shape:', X_train_padded.shape)
print('y_train_padded shape:', y_train_padded.shape)
print('X_test_padded shape:', X_test_padded.shape)
print('y_test_padded shape:', y_test_padded.shape)

X_train_padded shape: (320, 92079, 98)
y_train_padded shape: (320, 92079, 88)
X_test_padded shape: (10, 92079, 98)
y_test_padded shape: (10, 92079, 88)


#### Reshape data

In [12]:
X_train = X_train_padded.reshape(-1, X_train_padded.shape[2])
y_train = y_train_padded.reshape(-1, 88)
X_test = X_test_padded.reshape(-1, X_test_padded.shape[2])
y_test = y_test_padded.reshape(-1, 88)

In [13]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (29465280, 98)
y_train shape: (29465280, 88)
X_test shape: (920790, 98)
y_test shape: (920790, 88)


#### Save preprocessed data

In [14]:
save_preprocessed_data(X_train, y_train, '../Data/processed/stft/frame_size_1024/', 'train.h5')
save_preprocessed_data(X_test, y_test, '../Data/processed/stft/frame_size_1024/', 'test.h5')

Starting to save preprocessed data...
Data successfully saved to ../Data/processed/stft/frame_size_1024/train.h5
Starting to save preprocessed data...
Data successfully saved to ../Data/processed/stft/frame_size_1024/test.h5


### Create preprocessed dataset with frame size = 2048

In [4]:
train_mean, train_std_dev = calculate_stft_stats(train_data_dir, frame_size=2048, data_size=320)
X_train, y_train = process_files_using_stft(train_data_dir, train_labels_dir, generate_note_labels, mean=train_mean, std=train_std_dev, frame_size=2048, data_size=320)

test_mean, test_std_dev = calculate_stft_stats(test_data_dir, frame_size=2048, data_size=10)
X_test, y_test = process_files_using_stft(test_data_dir, test_labels_dir, generate_note_labels, mean=test_mean, std=test_std_dev, frame_size=2048, data_size=10)

Calculating STFT stats: 100%|██████████| 320/320 [02:04<00:00,  2.57it/s]
Processing files in '../Data/raw/musicnet/train_data' and '../Data/raw/musicnet/train_labels': 100%|██████████| 320/320 [03:15<00:00,  1.63it/s]
Calculating STFT stats: 100%|██████████| 10/10 [00:01<00:00,  6.66it/s]
Processing files in '../Data/raw/musicnet/test_data' and '../Data/raw/musicnet/test_labels': 100%|██████████| 10/10 [00:02<00:00,  4.29it/s]


#### Pad data to the same length

In [5]:
print('Number of training examples:', len(X_train))
print('Number of training labels:', len(y_train))
print(f"X_train[0] shape: {X_train[0].shape}")
print(f"y_train[0] shape: {y_train[0].shape}")

Number of training examples: 320
Number of training labels: 320
X_train[0] shape: (19254, 194)
y_train[0] shape: (19254, 88)


In [6]:
max_length = max(find_max_length(X_train), find_max_length(X_test))
print('Max length:', max_length)

Max length: 46040


In [ ]:
X_train_padded = pad_data(X_train, max_length)
y_train_padded = pad_data(y_train, max_length)
X_test_padded = pad_data(X_test, max_length)
y_test_padded = pad_data(y_test, max_length)

In [8]:
print('X_train_padded shape:', X_train_padded.shape)
print('y_train_padded shape:', y_train_padded.shape)
print('X_test_padded shape:', X_test_padded.shape)
print('y_test_padded shape:', y_test_padded.shape)

X_train_padded shape: (320, 46040, 194)
y_train_padded shape: (320, 46040, 88)
X_test_padded shape: (10, 46040, 194)
y_test_padded shape: (10, 46040, 88)


#### Reshape data

In [9]:
X_train = X_train_padded.reshape(-1, X_train_padded.shape[2])
y_train = y_train_padded.reshape(-1, 88)
X_test = X_test_padded.reshape(-1, X_test_padded.shape[2])
y_test = y_test_padded.reshape(-1, 88)

In [10]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (14732800, 194)
y_train shape: (14732800, 88)
X_test shape: (460400, 194)
y_test shape: (460400, 88)


#### Save preprocessed data

In [11]:
save_preprocessed_data(X_train, y_train, '../Data/processed/stft/frame_size_2048/', 'train.h5')
save_preprocessed_data(X_test, y_test, '../Data/processed/stft/frame_size_2048/', 'test.h5')

Starting to save preprocessed data...
Data successfully saved to ../Data/processed/stft/frame_size_2048/train.h5
Starting to save preprocessed data...
Data successfully saved to ../Data/processed/stft/frame_size_2048/test.h5
